In [1]:
import ROOT
import numpy as np
import pandas as pd
import math

Welcome to JupyROOT 6.16/00


In [2]:
!ls

ls.C		   test_WenTT.csv    test_ZeeTT.csv    test_ZnnTT.csv
ls.C~		   test_WenVj.csv    test_ZeeVj.csv    test_ZnnVj.csv
out.txt		   test_WenVV.csv    test_ZeeVV.csv    test_ZnnVV.csv
outWen.txt	   test_WenVVLF.csv  test_ZeeVVLF.csv  test_ZnnVVLF.csv
outZnn.txt	   test_Wmn.csv      test_Zmm.csv      vhbb_Wen-2017.root
sensitivity_2.py   test_WmnTT.csv    test_ZmmTT.csv    vhbb_Wmn-2017.root
sensitivity.ipynb  test_WmnVj.csv    test_ZmmVj.csv    vhbb_Zee-2017.root
sensitivity.py	   test_WmnVV.csv    test_ZmmVV.csv    vhbb_Zmm-2017.root
sensitivity.py~    test_WmnVVLF.csv  test_ZmmVVLF.csv  vhbb_Znn-2017.root
test_Wen.csv	   test_Zee.csv      test_Znn.csv


In [3]:
_verbos = 1

In [4]:
def AsimovSignificance(h_s, h_b, sigma):
    n_bins = h_s.GetNbinsX()
    #sigma_b = sigma
    #ar_AS = pd.DataFrame()
    ar_AS = []
    for n in range(n_bins):
        #print n 
        s = h_s.GetBinContent(n)
        b = h_b.GetBinContent(n)
        sigma_b = h_b.GetBinError(n)
        #print s
        #print b
        
        AS = ROOT.RooStats.AsimovSignificance(s, b, sigma_b)

        AS_0 = ROOT.RooStats.AsimovSignificance(s, b, 0)
        #print AS
        ar_AS_ = np.array([s, b, AS, sigma_b, AS_0])
        ar_AS.append(ar_AS_)
    return ar_AS

In [5]:
def Sensitivity(df, name):
    if _verbos > 1:
        print(df)
    df['AsimovSignificance2%s' %(name)] = df['AsimovSignificance%s' % (name)]**2 
    sen = df[(df['AsimovSignificance2%s' %(name)] >0) & (df['AsimovSignificance2%s' %(name)] < float('inf'))].sum()
    sen = sen['AsimovSignificance2%s' %(name)]
    
    df['AsimovSignificance2%s_0' %(name)] = df['AsimovSignificance%s_0' % (name)]**2 
    sen_0 = df[(df['AsimovSignificance2%s_0' %(name)] >0) & (df['AsimovSignificance2%s_0' %(name)] < float('inf'))].sum()
    sen_0 = sen_0['AsimovSignificance2%s_0' %(name)]
    
    return math.sqrt(sen), math.sqrt(sen_0)

In [13]:
def calculate(filename, channel, input_sigma, probe, n_, a, b):
    f_Zee = ROOT.TFile("%s" % (filename))
    h_s_low = ROOT.TH1F("h_s_low", "h_s_low", n_, a, b)
    h_s_med = ROOT.TH1F("h_s_med", "h_s_med", n_, a, b)
    h_s_med_ge1j = ROOT.TH1F("h_s_med_ge1j", "h_s_med_ge1j", n_, a, b)
    h_s_high = ROOT.TH1F("h_s_high", "h_s_high", n_, a, b)

    reg_nam = ['SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    sam_nam = ['WH_lep', 'ZH_lep', 'ggZH_lep']
    #sam_nam = ['ZH_lep', 'ggZH_lep']
    ptv = ['PTV_0_75_hbb', 'PTV_150_250_0J_hbb', 'PTV_150_250_GE1J_hbb', 'PTV_75_150_hbb', 'PTV_GT250_hbb']
    #ptv = ['PTV_0_75_hbb',   'PTV_75_150_hbb']

    for reg in reg_nam:
        for sam in sam_nam:
            if "%s" %(channel) not in reg: break
            for pt in ptv:
                if "low" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_low.Add(h_)
                if "med" in reg and "0j" in reg:
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med.Add(h_)
                if "med" in reg and "ge1j" in reg:
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med_ge1j.Add(h_)                    
                if "high" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_high.Add(h_)
    
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    sample_name = ['TT', 'VVHF', 'VVLF', 'Wj0b', 'Wj1b', 'Wj2b', 'Zj0b', 'Zj1b', 'Zj2b', 's_Top']
    h_b_low = ROOT.TH1F("h_b_low", "h_b_low", n_, a, b)
    h_b_med = ROOT.TH1F("h_b_med", "h_b_med", n_, a, b)
    h_b_med_ge1j = ROOT.TH1F("h_b_med_ge1j", "h_b_med_ge1j", n_, a, b)

    h_b_high = ROOT.TH1F("h_b_high", "h_b_high", n_, a, b)

    for region in region_name:
        for sample in sample_name:
            if "%s" %(channel) not in region: break
            if "low" in region:
                h_ = ROOT.TH1F()
                h_.Clear()
                h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                if probe in sample:
                    n_bns = h_.GetNbinsX()
                    for n in range(n_bns):
                        sgma_b = h_.GetBinError(n)
                        #print(sgma_b, sgma_b/math.sqrt(2.))
                        h_.SetBinError(n, sgma_b/math.sqrt(2.))
                        #print(sgma_b, h_.GetBinError(n))
                if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                h_b_low.Add(h_)
            if "high" in region:
                if channel=="Znn" and "Wj2b" in sample: continue 
                h_ = ROOT.TH1F()
                h_.Clear()
                h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                if probe in sample:
                    n_bns = h_.GetNbinsX()
                    for n in range(n_bns):
                        sgma_b = h_.GetBinError(n)
                        h_.SetBinError(n, sgma_b/math.sqrt(2.))                
                if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                h_b_high.Add(h_)
            if "med" in region and "0j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                if probe in sample:
                    n_bns = h_.GetNbinsX()
                    for n in range(n_bns):
                        sgma_b = h_.GetBinError(n)
                        h_.SetBinError(n, sgma_b/math.sqrt(2.))                
                if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                h_b_med.Add(h_)
            if "med" in region and "ge1j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                if probe in sample:
                    n_bns = h_.GetNbinsX()
                    for n in range(n_bns):
                        sgma_b = h_.GetBinError(n)
                        h_.SetBinError(n, sgma_b/math.sqrt(2.))                
                if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                h_b_med_ge1j.Add(h_)
            

    
    
    dict_AS = {}

    sigma_b = input_sigma
    region_names = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    for region in region_names:
        if "%s" %(channel) not in region: continue
        if _verbos > 1: print(region)
        #print("BDT_%s_TT" % (region))
        #print("BDT_%s_%s" % (region, sample))
        if "low" in region:
            h_s = h_s_low.Clone()
            h_b = h_b_low.Clone()
        if "high" in region:
            h_s = h_s_high.Clone()
            h_b = h_b_high.Clone()
        if "med" in region and "0j" in region:
            h_s = h_s_med.Clone()
            h_b = h_b_med.Clone()
        if "med" in region and "ge1j" in region:
            h_s = h_s_med_ge1j.Clone()
            h_b = h_b_med_ge1j.Clone()
        
                
        #h_b = f_Zee.Get("BDT_%s_%s" % (region, sample))
        l = AsimovSignificance(h_s, h_b, sigma_b)
        df = pd.DataFrame(l, columns=['signal', 'background', 'AsimovSignificance%s' %(region), 'sigma_b', 'AsimovSignificance%s_0' %(region)])
        sen, sen_0 = Sensitivity(df, region)
        dict_AS.update({region : [sen, sen_0 ]})

    df_AS = pd.DataFrame.from_dict(dict_AS, orient='index', columns=['AsimovSignificance', 'AsimovSignificance_0'])

    return df_AS

Zee AMS:

In [15]:
_verbos=1
test_Zee = calculate("vhbb_Zee-2017.root", "Zee", 0, "aaa", 20, -1, 1)
test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
#print(test_Zee)
test_Zee.to_csv("test_Zee.csv")
!cat test_Zee.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zee,0.6322607852681823,0.6789413012018976,0.06875486268264863
SR_med_Zee_ge1j,0.5554824100621002,0.5920300987482715,0.0617328219687547
SR_med_Zee_0j,0.7999109914493602,0.8584333317152688,0.068173424893664
SR_low_Zee,0.5900068004039632,0.7082241224187588,0.1669207787092233


In [16]:
test_Zee = calculate("vhbb_Zee-2017.root", "Zee", 0, "TT", 20, -1, 1)
test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
#print(test_Zee)
test_Zee.to_csv("test_ZeeTT.csv")
!cat test_ZeeTT.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zee,0.6322965314706867,0.6789413012018976,0.06870221276660866
SR_med_Zee_ge1j,0.5568238301300666,0.5920300987482715,0.05946702489052741
SR_med_Zee_0j,0.8007115146647419,0.8584333317152688,0.06724088513104533
SR_low_Zee,0.5930104676408493,0.7082241224187588,0.1626796534187887


In [17]:
test_Zee = calculate("vhbb_Zee-2017.root", "Zee", 0, "VV", 20, -1, 1)
test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
#print(test_Zee)
test_Zee.to_csv("test_ZeeVV.csv")
!cat test_ZeeVV.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zee,0.6361663048250984,0.6789413012018976,0.06300249565179879
SR_med_Zee_ge1j,0.5581352741817585,0.5920300987482715,0.05725186040063982
SR_med_Zee_0j,0.8028267178962821,0.8584333317152688,0.06477685775303826
SR_low_Zee,0.5908311824491181,0.7082241224187588,0.16575676576606144


In [18]:
test_Zee = calculate("vhbb_Zee-2017.root", "Zee", 0, "VVLF", 20, -1, 1)
test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
#print(test_Zee)
test_Zee.to_csv("test_ZeeVVLF.csv")
!cat test_ZeeVVLF.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zee,0.6323917039037503,0.6789413012018976,0.06856203506215144
SR_med_Zee_ge1j,0.5555201904389752,0.5920300987482715,0.06166900700908484
SR_med_Zee_0j,0.79998045197544,0.8584333317152688,0.06809250943580192
SR_low_Zee,0.5900190317348472,0.7082241224187588,0.1669035082852195


In [19]:
test_Zee = calculate("vhbb_Zee-2017.root", "Zee", 0, "j", 20, -1, 1)
test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
#print(test_Zee)
test_Zee.to_csv("test_ZeeVj.csv")
!cat test_ZeeVj.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zee,0.6498118119071743,0.6789413012018976,0.04290428236308014
SR_med_Zee_ge1j,0.5683976865339366,0.5920300987482715,0.03991758571785599
SR_med_Zee_0j,0.8232523690396396,0.8584333317152688,0.040982754718217614
SR_low_Zee,0.6330414029329852,0.7082241224187588,0.10615667711092105


Zmm AMS:

In [20]:
test_Zmm = calculate("vhbb_Zmm-2017.root", "Zmm", 0, "aaa", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_Zmm.csv")
!cat test_Zmm.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.7236043043843104,0.7772729317231171,0.06904733864825316
SR_med_Zmm_ge1j,0.6448617714079454,0.6950647061668356,0.07222771392860797
SR_med_Zmm_0j,0.9227698737397871,0.987916516848541,0.06594346991643786
SR_low_Zmm,0.7242509677970452,0.868058059138925,0.16566529142593295


In [21]:
test_Zmm = calculate("vhbb_Zmm-2017.root", "Zmm", 0, "TT", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_ZmmTT.csv")
!cat test_ZmmTT.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.7237553686818818,0.7772729317231171,0.06885298697150494
SR_med_Zmm_ge1j,0.6467140486359787,0.6950647061668356,0.06956281494639914
SR_med_Zmm_0j,0.9243318386196564,0.987916516848541,0.06436240020737789
SR_low_Zmm,0.7281239055293004,0.868058059138925,0.16120368002623356


In [22]:
test_Zmm = calculate("vhbb_Zmm-2017.root", "Zmm", 0, "VV", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_ZmmVV.csv")
!cat test_ZmmVV.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.7281129761204621,0.7772729317231171,0.06324670987020412
SR_med_Zmm_ge1j,0.64787847384911,0.6950647061668356,0.06788753895727156
SR_med_Zmm_0j,0.9253234564750549,0.987916516848541,0.06335865359672121
SR_low_Zmm,0.7253216922697894,0.868058059138925,0.1644318203908201


In [23]:
test_Zmm = calculate("vhbb_Zmm-2017.root", "Zmm", 0, "VVLF", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_ZmmVVLF.csv")
!cat test_ZmmVVLF.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.7239820808076615,0.7772729317231171,0.06856131062909451
SR_med_Zmm_ge1j,0.6449237753384364,0.6950647061668356,0.07213850794542277
SR_med_Zmm_0j,0.9228400254142445,0.987916516848541,0.06587246019723494
SR_low_Zmm,0.7242735037814846,0.868058059138925,0.16563933004673476


In [24]:
test_Zmm = calculate("vhbb_Zmm-2017.root", "Zmm", 0, "j", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_ZmmVj.csv")
!cat test_ZmmVj.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.7439048525701141,0.7772729317231171,0.04292968118551373
SR_med_Zmm_ge1j,0.6630084790227099,0.6950647061668356,0.04611977397170744
SR_med_Zmm_0j,0.9488391393241543,0.987916516848541,0.03955534385541373
SR_low_Zmm,0.7785468307058333,0.868058059138925,0.10311663775333511


Wen AMS:

In [25]:
test_Wen = calculate("vhbb_Wen-2017.root", "Wen", 0, "aaa", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_Wen.csv")
!cat test_Wen.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.2635508952013455,0.2956839751666351,0.10867372825051044
SR_med_Wen_0j,0.9158982713578641,1.0301115529789004,0.11087467303007308
SR_high_Wen,0.9488430560652082,1.0324724111996086,0.08099911845318283


In [26]:
test_Wen = calculate("vhbb_Wen-2017.root", "Wen", 0, "TT", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_WenTT.csv")
!cat test_WenTT.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.2740785457944522,0.2956839751666351,0.07306932802160479
SR_med_Wen_0j,0.9351361559321311,1.0301115529789004,0.09219913782358555
SR_high_Wen,0.9626951663067066,1.0324724111996086,0.0675826725595789


In [27]:
test_Wen = calculate("vhbb_Wen-2017.root", "Wen", 0, "VV", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_WenVV.csv")
!cat test_WenVV.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.2635669298208357,0.2956839751666351,0.10861949934114482
SR_med_Wen_0j,0.9159688620431478,1.0301115529789004,0.11080614580592967
SR_high_Wen,0.9489635197147108,1.0324724111996086,0.08088244352008452


In [28]:
test_Wen = calculate("vhbb_Wen-2017.root", "Wen", 0, "VVLF", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_WenVVLF.csv")
!cat test_WenVVLF.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.26355388405486696,0.2956839751666351,0.10866361998028791
SR_med_Wen_0j,0.915905168850569,1.0301115529789004,0.11086797716040242
SR_high_Wen,0.9488430560651311,1.0324724111996086,0.08099911845325757


In [29]:
test_Wen = calculate("vhbb_Wen-2017.root", "Wen", 0, "j", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_WenVj.csv")
!cat test_WenVj.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.2654193539326949,0.2956839751666351,0.10235462106759191
SR_med_Wen_0j,0.9316259790709344,1.0301115529789004,0.09560670747082024
SR_high_Wen,0.9642207924394437,1.0324724111996086,0.066105029073721


Wmn AMS:

In [30]:
test_Wmn = calculate("vhbb_Wmn-2017.root", "Wmn", 0, "aaa", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_Wmn.csv")
!cat test_Wmn.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wmn_ge1j,0.32182262494928565,0.3613190740413683,0.10931182970860989
SR_med_Wmn_0j,1.1762677932132863,1.2944830714163442,0.0913223825118964
SR_high_Wmn,1.0040708481092404,1.1346675321666178,0.1150968723040893


In [31]:
test_Wmn = calculate("vhbb_Wmn-2017.root", "Wmn", 0, "TT", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_WmnTT.csv")
!cat test_WmnTT.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wmn_ge1j,0.3343310600362577,0.3613190740413683,0.07469302326956787
SR_med_Wmn_0j,1.1986095949661406,1.2944830714163442,0.07406313652700357
SR_high_Wmn,1.0226111061025867,1.1346675321666178,0.09875705692403335


In [32]:
test_Wmn = calculate("vhbb_Wmn-2017.root", "Wmn", 0, "VV", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_WmnVV.csv")
!cat test_WmnVV.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wmn_ge1j,0.32183715107775795,0.3613190740413683,0.10927162665951584
SR_med_Wmn_0j,1.1769750993991746,1.2944830714163442,0.09077598202083835
SR_high_Wmn,1.0045000508858248,1.1346675321666178,0.11471860927600663


In [33]:
test_Wmn = calculate("vhbb_Wmn-2017.root", "Wmn", 0, "VVLF", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_WmnVVLF.csv")
!cat test_WmnVVLF.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wmn_ge1j,0.3218261141555109,0.3613190740413683,0.10930217285272847
SR_med_Wmn_0j,1.1762728611133275,1.2944830714163442,0.0913184675128106
SR_high_Wmn,1.004128240107395,1.1346675321666178,0.1150462918507604


In [34]:
test_Wmn = calculate("vhbb_Wmn-2017.root", "Wmn", 0, "j", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_WmnVj.csv")
!cat test_WmnVj.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wmn_ge1j,0.32451300398297617,0.3613190740413683,0.1018658374348046
SR_med_Wmn_0j,1.1932983943253646,1.2944830714163442,0.07816608754896237
SR_high_Wmn,1.0340211898723852,1.1346675321666178,0.08870117408052647


Znn AMS:

In [35]:
_verbos = 1
test_Znn = calculate("vhbb_Znn-2017.root", "Znn", 0,"aaa", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_Znn.csv")
!cat test_Znn.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.1481556583618754,0.16160600124525362,0.08322922898739364
SR_med_Znn_0j,0.25934823643288685,0.27867629989587456,0.06935668182120082
SR_high_Znn,0.1940963570988533,0.2093053414823378,0.07266410057082984


In [36]:
test_Znn = calculate("vhbb_Znn-2017.root", "Znn", 0,"TT", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_ZnnTT.csv")
!cat test_ZnnTT.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.15192434117623954,0.16160600124525362,0.05990903799618909
SR_med_Znn_0j,0.26283716021924386,0.27867629989587456,0.05683705317800215
SR_high_Znn,0.19666149666972274,0.2093053414823378,0.06040861032532223


In [37]:
test_Znn = calculate("vhbb_Znn-2017.root", "Znn", 0,"VV", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_ZnnVV.csv")
!cat test_ZnnVV.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.1483714893452771,0.16160600124525362,0.08189369081592329
SR_med_Znn_0j,0.26013433304802114,0.27867629989587456,0.0665358584665488
SR_high_Znn,0.19536985064186452,0.2093053414823378,0.06657971909259285


In [38]:
test_Znn = calculate("vhbb_Znn-2017.root", "Znn", 0,"VVLF", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_ZnnVVLF.csv")
!cat test_ZnnVVLF.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.1481565936741278,0.16160600124525362,0.08322344137898047
SR_med_Znn_0j,0.25936547149474093,0.27867629989587456,0.06929483565107253
SR_high_Znn,0.19409831837397656,0.2093053414823378,0.07265473016915082


In [39]:
test_Znn = calculate("vhbb_Znn-2017.root", "Znn", 0,"j", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_ZnnVj.csv")
!cat test_ZnnVj.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.14991192333466702,0.16160600124525362,0.07236165625334448
SR_med_Znn_0j,0.26316431442608257,0.27867629989587456,0.055663095410653625
SR_high_Znn,0.19671635310828592,0.2093053414823378,0.06014652222869424
